In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

hbar = 1
m = 1
N = 400

try:
    from numpy import heaviside
    heaviside = heaviside
except:
    heaviside = lambda x,a: (x > 0) * 1

# Some does not have scipy.linalg.eigh_tridiagonal, so this takes care of that
# TODO: leave only one in the end
try:
    from scipy.linalg import eigh_tridiagonal
    get_eigh = lambda H: eigh_tridiagonal(np.diag(H), np.diag(H, k=1))
except ImportError:
    get_eigh = np.linalg.eigh

def _interval(x, a, b):
    """One in interval [a,b], 0 otherwise"""
    return heaviside(x - a, 1) * heaviside(b - x, 1)

def V_crystal(x, N_w = 5, b=0.1):
    """Poly well potential
    N_w  : number of wells
    b    : width between wells"""
    w = (1.0 - (N_w - 1) * b)/(N_w + 20.0) # Add 20 because we want to have 10w space on either side
    pos = (x - 10*w) % (w + b)
    return V_atom(pos, pos=0, w=w) * _interval(x, 10*w, 1 - 10*w)
    
def investigate_potential(V, N, states = range(5), **kwargs):
    dx = 1 / (N + 1)
    x = np.linspace(dx, N * dx, N)
    v = V(x, **kwargs)

    H = np.zeros((N, N))
    for i in range(0, N):
        H[i][i] = hbar**2 / (m * dx**2) + v[i]
    for i in range(0, N - 1):
        H[i][i+1] = -hbar**2 / (2 * m * dx**2)
        H[i+1][i] = -hbar**2 / (2 * m * dx**2)
    
    eigvals, eigvecs = get_eigh(H)
    energies = eigvals
    waves = eigvecs.T

    prod = eigvecs.T @ eigvecs
    print("Greatest deviation from orthonormality:")
    print(np.max(np.abs(eigvecs.T @ eigvecs - np.eye(N))))

    plt.figure(figsize=(16, 8))
    plt.title(V.__doc__.split('\n')[0] + ", discrete, N = %d points" % N)
    plt.xlabel("$x / L$")
    plt.ylabel("$V(x), E_n$")
    plt.plot(x, v, "grey", linestyle="dashed", linewidth=2, label="$V(x)$")
    for i in range(0, len(states)):
        n = states[i]
        energy = energies[n]
        wave = waves[n]
        plt.plot((0, 1), (energy, energy), "C%d:" % (i % 10), linewidth=1)
        plt.plot(x, energy + wave * 1000, "C%d-" % (i % 10), linewidth=1, label="$E_{%d}, \psi_{%d}$" % (n, n))
    plt.legend(loc="upper left")
    plt.show()

investigate_potential(V_crystal, N, N_w=0)
investigate_potential(V_crystal, N, N_w=1)
investigate_potential(V_crystal, N, N_w=2)
investigate_potential(V_crystal, N, N_w=5)

# Oppgave 2

### a)
Figuren viser en potensialbrønn med utvalgte tilhørende bølgefunksjoner. Brønnens lengde og bredde er her valgt slik at det finnes tre bølgefunksjoner innenfor brønnen. Fra figuren kommer det frem at annenhver bølgeunksjon er symmetrisk og antisymmetrisk, hvilket stemmer med teori. Man serr også at bølgefunksjonene for grunntilstand, første og andre eksiterte tilstand henholdsvis har 0, 1 og 2 nullpunkter. De oppfyller også initialbetingelsene om at de skal være kontinuerlige og at de går mot null i stor avstand fra brønnen, når potensialet er null.

Denne potensialfunksjonen kan modellere et atom, og bølgefunksjonene er da mulige tilstander elektroner kan befinne seg i. 
### TODO: Sjekk energiegenverdiene opp mot analytiske verdier.

Det er også tegnet inn en bølgefunksjon ovenfor brønnen. Denne representerer bølgefunksjonen for et ubundet elektron (fritt elektron), som har for stor kinetisk energi til å fanges av det elektriske feltet til atomkjernen. Et slikt elektron skal, langt unna atomet, ha en upåvirket, sinusoidal bølgefunksjon. I figuren er dette tilfelle langt ut til sidene. 

Her kan bølgelengden til elektronet leses av fra grafen.
### TODO Les av bølgelengden. Si litt mer om elektronet?

### b)
Med tre bundne tilstander er det plass til inntil seks elektroner innenfor potensialbrønnen. Dette fordi Pauliprinsippet utelukker muligheten for at det kan eksistere mer enn to elektroner (ett med spin opp og ett med spinn ned) per bundne tilstand. En slik potensialbrønn kan modellere atomnummer fem og seks (bor og karbon), som har fem og seks elektroner i grunntilstand. Deres totale elektronspinn finnes fra Pauliprinsippet; elektronene i karbonatomets tre elektronpar må ha motsatt spinn, slik at det samlede elektronspinnet til $C^0$ må være null. Samme argumentasjon for boratomet, som har fem elektroner, gir rom for at det femte elektronet (som med stor sannsynlighet befinner seg i den mest energirike andre eksiterte tilstanden) kan ha spinn op eller ned, slik at atomets totale elektronspinn er $S = \pm \frac{\sqrt{3}}{2} \hbar$